# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 0.Setting Colab Method for future model developing
Firstly, run the following block to mount the drive to the colab. Then, drag the data folder/**eval.py** to the "Colab Folder Space" to ensure the code runs successfully.

If data folder updated, attempt to forcibly remount, call `drive.mount("/content/drive", force_remount=True)`.


In [66]:
#from google.colab import drive
#drive.mount('/content/drive')

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [67]:
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from statistics import mean
from collections import Counter

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [68]:
with open('data/train-claims.json', 'r') as input_file:
    train_claim_data = json.load(input_file)

# Read in development data (claim)
with open('data/dev-claims.json', 'r') as input_file:
    dev_claim_data = json.load(input_file)

# Read in test data (claim)
with open('data/test-claims-unlabelled.json', 'r') as input_file:
    test_claim_data = json.load(input_file)

# Read in evidence data
with open('data/evidence.json', 'r') as input_file:
    evi_data = json.load(input_file)

#EDA




claim_count = 0
evi_count = 0
claim_length = []
evidence_count = []
evidence_length = []
labels = []

for key,value in train_claim_data.items():
    claim_count+=1
    claim_length.append(len(value["claim_text"]))
    evidence_count.append(len(value["evidences"]))
    evidence_length += [len(evi_data[x]) for x in value["evidences"]]
    labels.append(value["claim_label"])

for key,value in evi_data.items():
    evi_count+=1

print("claim count: ",claim_count)
print("evidence count: ",evi_count)
print("max claim length: ",max(claim_length))
print("min claim length: ",min(claim_length))
print("mean claim length: ",mean(claim_length))
print("max evidence count: ",max(evidence_count))
print("min evidence count: ",min(evidence_count))
print("mean evidence count: ",mean(evidence_count))
print("max evidence length: ",max(evidence_length))
print("min evidence length: ",min(evidence_length))
print("mean evidence length: ",mean(evidence_length))
print(Counter(labels))



inside = 0
outside = 0

train_evi_id = []
for claim_id,claim_value in train_claim_data.items():
    train_evi_id=train_evi_id+claim_value['evidences']

for claim_id,claim_value in dev_claim_data.items():
    test_evi_id=claim_value['evidences']
    for e in test_evi_id:
        if e in train_evi_id:
            inside +=1
        else:
            outside += 1
print("Dev evi inside train evi", inside)
print("Dev evi outside train evi", outside)

full_evidence_id = list(evi_data.keys())
full_evidence_text  = list(evi_data.values())
train_claim_id = list(train_claim_data.keys())
train_claim_text  = [ v["claim_text"] for v in train_claim_data.values()]


claim count:  1228
evidence count:  1208827
max claim length:  332
min claim length:  26
mean claim length:  122.95521172638436
max evidence count:  5
min evidence count:  1
mean evidence count:  3.3566775244299674
max evidence length:  1979
min evidence length:  13
mean evidence length:  173.5
Counter({'SUPPORTS': 519, 'NOT_ENOUGH_INFO': 386, 'REFUTES': 199, 'DISPUTED': 124})
Dev evi inside train evi 163
Dev evi outside train evi 328


Remove unnecessary data

In [69]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [76]:
print(full_evidence_id[0:10])

['evidence-0', 'evidence-1', 'evidence-2', 'evidence-3', 'evidence-4', 'evidence-5', 'evidence-6', 'evidence-7', 'evidence-8', 'evidence-9']


In [70]:
import string
# remove non-english words
def is_pure_english(text):
    english_letters = set(string.ascii_letters)
    cleaned_text = ''.join(char for char in text if char.isalpha() or char.isspace())
    return all(char in english_letters or char.isspace() for char in cleaned_text)

english_evidence = [sentence for sentence in full_evidence_text if is_pure_english(sentence)]
count_word_eng = 0
count_word = 0
for evi in english_evidence:
    count_word_eng+=1
for evi in full_evidence_text:
    count_word+=1
print("word count: ",count_word)
print("english word count: ",count_word_eng)
print("word deleted: ",count_word-count_word_eng)
print("word deleted percentage: ",(count_word-count_word_eng)/count_word)
print(english_evidence[0:10])
print(full_evidence_text[0:10])

word count:  1208827
english word count:  1114577
word deleted:  94250
word deleted percentage:  0.07796814597953222
['John Bennet Lawes, English entrepreneur and agricultural scientist', 'Lindberg began his professional career at the age of 16, eventually moving to New York City in 1977.', "``Boston (Ladies of Cambridge)'' by Vampire Weekend", 'Gerald Francis Goyer (born October 20, 1936) was a professional ice hockey player who played 40 games in the National Hockey League.', 'He detected abnormalities of oxytocinergic function in schizoaffective mania, post-partum psychosis and how ECT modified oxytocin release.', 'With peak winds of 110 mph (175 km/h) and a minimum pressure of 972 mbar (hPa ; 28.71 inHg), Florence was the strongest storm of the 1994 Atlantic hurricane season.', 'He is currently a professor of piano at the University of Wisconsin -- Madison since August 2000.', 'In addition to known and tangible risks, unforeseeable black swan extinction events may occur, presenting

In [78]:
def find_keys(dictionary, target_list):
    keys = []
    for key, value in dictionary.items():
        if value in target_list:
            keys.append(key)
    return keys

english_evidence_id = find_keys(evi_data, english_evidence)
print(len(english_evidence_id))

KeyboardInterrupt: 

In [71]:

# dev remove stop word会好一点， test不remove反而好
evidence_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)

# claim_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)
claim_tfidf_vectorizer = TfidfVectorizer(use_idf=True)


evidence_tfidf_vectorizer.fit(train_claim_text+full_evidence_text)
train_claim_emb_list = claim_tfidf_vectorizer.fit_transform(train_claim_text)


full_evi_emb_list = evidence_tfidf_vectorizer.transform(full_evidence_text)


In [72]:
print(full_evi_emb_list.shape)


(1208827, 605251)


In [73]:

evi_k=2
claim_k=1

with open('data/dev-claims.json', 'r') as input_file:
    test_out_temp = json.load(input_file)

for claim_id,claim_value in test_out_temp.items():
    # Task1
    # 把test claim转化成vector

    test_claim_emb = evidence_tfidf_vectorizer.transform([claim_value['claim_text']])
    evi_sim_dict = {}
    # 计算出test claim和所有evidence的相似度
    sim = cosine_similarity(test_claim_emb, full_evi_emb_list)[0]

    for i in range(len(sim)):
        evi_sim_dict[full_evidence_id[i]] = sim[i]

    # 对evidence根据和claim的相似度排序
    s_sim = [(k, v) for k, v in sorted(evi_sim_dict.items(), key=lambda item: item[1],reverse=True)][:evi_k]
    sel_sim = [k for k,v in s_sim]
    # 把最相似的前k个evidence的id写入到test claim的evidence list
    test_out_temp[claim_id]["evidences"] = sel_sim

    # Task2
    # 把test claim转化成vector 注意我这两问用了不同的vectorizer，因为有不同的预处理步骤，和应用数据目标
    test_claim_emb = claim_tfidf_vectorizer.transform([claim_value['claim_text']])

    # 计算出test claim和所有train claim的相似度
    claim_sim_dict = {}
    claim_sim = cosine_similarity(test_claim_emb, train_claim_emb_list)[0]
    for i in range(len(claim_sim)):
        claim_sim_dict[train_claim_id[i]] = claim_sim[i]

    # 取最相似的k个train claim
    most_sim_claims = [(k, v) for k, v in sorted(claim_sim_dict.items(), key=lambda item: item[1],reverse=True)]
    # 我这里用的是k=1只考虑最相似的那一个，label拿出来
    most_sim_claim = max(most_sim_claims, key=operator.itemgetter(1))[0]

    test_out_temp[claim_id]["claim_label"] = train_claim_data[most_sim_claim]["claim_label"]



# Writing to sample.json
with open("data/dev_predict.json", "w") as outfile:
    json.dump(test_out_temp, outfile)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [74]:
# %%cmd
# python eval.py --predictions dev-claims-baseline.json --groundtruth dev-claims.json
# python eval.py --predictions dev_predict.json --groundtruth dev-claims.json


import subprocess

# proc = subprocess.Popen(["python", "eval.py", "--predictions", "data\dev_predict.json", "--groundtruth", "data\dev-claims.json"
# ], stdout=subprocess.PIPE, shell=True)
# (out, err) = proc.communicate()
# print(str(out))

# 高自动化模型/预处理选择，可以自动读取准确度
output = subprocess.check_output("python eval.py --predictions data/dev_predict.json --groundtruth data/dev-claims.json", shell=True)

print(output)

b'Evidence Retrieval F-score (F)    = 0.10510204081632651\r\nClaim Classification Accuracy (A) = 0.474025974025974\r\nHarmonic Mean of F and A          = 0.17205555936935077\r\n'


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*